# Sesión 1 — SOLUCIÓN (Solo instructor)

Solución de referencia para el instructor.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../data/telco_churn.csv')
df.head()

## Actividad 1 — EDA técnico

In [ ]:
# 1. Carga y validación
df.info()
df.describe(include='all')
df.nunique()
df.dtypes

In [ ]:
# TotalCharges viene como object por strings vacíos
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.info()

In [ ]:
# 2. Calidad de datos
missing_pct = df.isnull().sum() / len(df) * 100
missing_pct[missing_pct > 0]

In [ ]:
# Outliers con IQR
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)]
    print(f'{col}: {len(outliers)} outliers')

In [ ]:
# Asimetría y curtosis
df[numeric_cols].skew()
df[numeric_cols].kurtosis()

In [ ]:
# 3. Univariado/bivariado
df[numeric_cols].hist(bins=30, figsize=(12, 4))
plt.tight_layout()
plt.show()

In [ ]:
df[numeric_cols].boxplot(figsize=(10, 4))
plt.xticks(rotation=0)
plt.show()

In [ ]:
df['Churn'].value_counts()
df['Contract'].value_counts()

In [ ]:
# Correlación Pearson + heatmap
corr = df[numeric_cols].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Matriz de correlación')
plt.tight_layout()
plt.show()

## Actividad 2 — Tipología de preguntas

| Pregunta | Tipo | Técnica | Justificación |
|----------|------|---------|---------------|
| ¿Cuál es la tasa de churn por región/contrato? | Descriptiva | SQL/BI | Agregación |
| ¿Por qué churnan más los de contrato mensual? | Diagnóstica | Chi-cuadrado, regresión logística | Asociación/causalidad |
| ¿Qué clientes churnarán en 3 meses? | Predictiva | ML supervisado | Predicción |
| ¿Qué oferta enviar para maximizar retención? | Prescriptiva | Optimización, bandits | Decisión óptima |

## Actividad 3 — KPIs

| KPI | Fórmula | Tipo | Modelable | Decisión |
|-----|---------|------|-----------|----------|
| Tasa de churn | clientes_perdidos_t / activos_t-1 | Lagging | No | Monitoreo |
| P(churn|X) | Score de riesgo | Leading | Sí | Intervención |
| LTV | Σ R_i/(1+r)^i | Leading | Sí | Priorización |

## Actividad 4 — Error conceptual

1. **Definición:** "Último mes" ambiguo (calendario vs rolling)
2. **Temporal:** Sesgo de supervivencia si usamos snapshot actual
3. **Leakage:** TotalCharges o EndDate usan info post-decisión

**Modelo erróneo:** Clasificador con TotalCharges → accuracy inflada, falla en producción